# SLB Test Tool

## Setup

### Install Selenium

In [6]:
from IPython.display import clear_output

%pip install selenium 
clear_output(wait=True)

%pip install webdriver_manager 
clear_output(wait=True)

%pip install beautifulsoup4
clear_output(wait=True)

%pip install pandas 
clear_output(wait=True)

%pip install ipywidgets
clear_output()

print("All packages installed")

All packages installed


In [7]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
from collections import defaultdict
from xml.sax.saxutils import quoteattr

import ipywidgets as widgets
from ipywidgets import HTML

import pandas as pd
from IPython.display import display
import time


prefs = {"profile.default_content_setting_values.notifications": 1}
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.quit()

clear_output()
print("Done!")

Done!


## Source Code

### Generate Objects Keys

In [39]:
def generate_key(tag, element, counter):
    base = {"input": "txt", "button": "btn", "a": "lnk", "img": "img"}
    prefix = base.get(tag, "elem")

    parts = [prefix]
    
    txt = element.get_text(strip=True)
    if txt:
        parts.append(txt.replace(" ", "_"))
        return key_builder(parts, counter)
    
    if element.get('name'):
        parts.append(element['name'])
        return key_builder(parts, counter)
    
    if element.get('aria-label'):
        parts.append(element['aria-label'].replace(" ", "_"))
        return key_builder(parts, counter)

    if element.get('id'):
        parts.append(element['id'])
        return key_builder(parts, counter)
    
    if element.get('data-test-id'):
        parts.append(element['data-test-id'])
        return key_builder(parts, counter)
    
    return key_builder(parts, counter)


def key_builder(parts, counter):
    key = '_'.join(filter(None, parts)).lower().replace("-", "_")
    counter[key] += 1
    return key if counter[key] == 1 else f"{key}_{counter[key]}"

print("Done!")

Done!


### Generate Objects Selectors

In [9]:
def generate_selector(element, driver):
    selectors = []

    for attr in ['id', 'name', 'placeholder', 'aria-label', 'data-test-id', 'alt']:
        if element.get(attr):
            value = quoteattr(element[attr])
            selectors.append(f"@{attr}={value}")


    if selectors:
        xpath_selector = f"//{element.name}[{' and '.join(selectors)}]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    txt = element.get_text(strip=True)
    if txt:
        xpath_selector = f"//{element.name}[contains(text(), {quoteattr(txt)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    if element.get('class'):
        classes = '.'.join(element.get('class'))
        xpath_selector = f"//{element.name}[contains(@class, {quoteattr(classes)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector

    return None

print("Done!")

Done!


### GUI

In [40]:
global df
global test_window
test_window = None


def new_window(url, wait=0):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(wait)
    return driver


def read_page_objects_metadata(driver):

    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    elements = soup.find_all(['input', 'button', 'a', 'img'])

    counter = defaultdict(int)
    result = defaultdict(list)

    for element in elements:
        tag = element.name
        selector = generate_selector(element, driver)
        key = generate_key(tag, element, counter)

        result[tag].append({
            "key": key,
            "selector": selector,
            "attributes": {attr: element.get(attr) for attr in element.attrs},
            "text": element.get_text(strip=True),
            "visibility": "visible" if element.get('type') != 'hidden' else "invisible"
        })
    return result



def highlight_element(driver, selector):
    script = f"""
    var element = document.evaluate('{selector}', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
    if (element) {{
        var originalStyle = element.style.border;
        var count = 0;
        var interval = setInterval(function() {{
            count += 1;
            element.style.border = count % 2 ? '3px solid red' : '';
            if (count > 9) {{
                clearInterval(interval);
                element.style.border = originalStyle;
            }}
        }}, 500);
    }}
    """
    try:
        driver.execute_script(script)
    except Exception as e:
        print("Error while highlighting the element:", e)



def start_playground():

    global test_window, df
    df = pd.DataFrame(columns=["Alias", "Key", "Selector", "Tag Name", "Visibility", "Select"])
    df_original = pd.DataFrame()
    selected_rows = []


    def on_new_window_clicked(b=None):
        global test_window
        try: 
            test_window.quit()
        except: 
            pass  # ignore exceptions
        test_window = new_window("https://life.stg.wellzesta.com/login")


    def on_metadata_clicked(b):
        clear_output(wait=True)
        global test_window, df
        if test_window is None:
            on_new_window_clicked()

        metadata = read_page_objects_metadata(test_window)
        rows = []
        for tag_name, elements in metadata.items():
            for element in elements:
                v = element.get('visibility', '')=="visible"
                rows.append({
                    "Export": False,
                    "Visible": v,
                    "Type": tag_name,
                    "Alias": element.get('key', ''),
                    "Web": element.get('selector', '')
                })

        df = pd.DataFrame(rows)
        update_table()
        print(f"Loaded from {test_window.current_url}")


    def on_export_button_clicked(b):
        global df
        df = df[df['Select'] == True]
        update_table()


    def on_filter_text_changed(change):
        global df
        filter_text = change.new.lower()
        if filter_text:
            df = df_original[df_original.apply(lambda row: filter_text in str(row).lower(), axis=1)]
        else:
            df = df_original
        update_table()


    def adjust_column_widths(df, base_width=10, padding=5):
        widths = {}
        for col in df.columns:
            max_len = df[col].astype(str).map(len).max()  # Comprimento máximo do conteúdo na coluna
            widths[col] = min(max_len * base_width + padding, 300)  # Calcular a largura com um máximo para evitar colunas muito largas
        return widths
    

    def on_select_row(change, index):
        global df
        df.at[index, 'Export'] = change['new']
        update_table()


    def update_table():
        global df, test_window
        nonlocal export_button

        if df.empty:
            clear_output(wait=True)
            display(widgets.HBox([new_window_button, metadata_button]))
            return

        df = df.drop(columns=["Attributes", "Text"], errors='ignore')
        df.sort_values(by=['Export', 'Type', 'Alias'], ascending=[False, True, True], inplace=True)

        clear_output(wait=True)
        display(widgets.HBox([new_window_button, metadata_button, filter_text, export_button]))

        # Calcular a largura das outras colunas
        col_widths = adjust_column_widths(df)
        grid = widgets.GridspecLayout(len(df) + 1, len(df.columns) + 1)

        # Cabeçalhos
        for i, col in enumerate(df.columns):
            grid[0, i] = HTML(value=f"<b>{col}</b>", layout=widgets.Layout(width=f'{col_widths[col]}px'))

        # Dados, botões de exportação e botões com seletores
        for i, (index, row) in enumerate(df.iterrows(), start=1):
            for j, col in enumerate(df.columns):
                if col == "Export":
                    export_button_text = 'Y' if row[col] else 'N'
                    export_button = widgets.Button(description=export_button_text, layout=widgets.Layout(width='30px'))
                    export_button.on_click(lambda b, index=index: on_select_row({'new': not row[col]}, index))
                    grid[i, j] = export_button

                elif col == "Web":
                    button_icon = 'fa-eye' 
                    selector_text = row[col] if row[col] else 'No Selector'
                    disabled = False if row[col] else True
                    highlight_button = widgets.Button(
                        disabled=disabled,
                        description='', 
                        tooltip=selector_text, 
                        icon=button_icon, 
                        layout=widgets.Layout(width='30px', height='30px')
                    )
                    highlight_button.on_click(lambda b, selector=row['Web']: highlight_element(test_window, selector))
                    grid[i, j] = highlight_button

                else:
                    grid[i, j] = widgets.Label(value=str(row[col]), layout=widgets.Layout(width=f'{col_widths[col]}px'))

        display(grid)


    # Widgets
    new_window_button = widgets.Button(description="New Window")
    metadata_button = widgets.Button(description="Inspect")
    filter_text = widgets.Text(placeholder='Type to filter...')
    export_button = widgets.Button(description="Export Selected")
    

    # Eventos
    new_window_button.on_click(on_new_window_clicked)
    metadata_button.on_click(on_metadata_clicked)
    filter_text.observe(on_filter_text_changed, names='value')
    export_button.on_click(on_export_button_clicked)


    # Inicializa a interface sem dados
    update_table()
    


start_playground()

/var/folders/hm/2613d1tx3dx75fn21zzrzn500000gq/T/ipykernel_20857/1279385580.py:168: DeprecationWarning: icons names no longer need 'fa-', just use the class names themselves (for example, 'gear spin' instead of 'fa-gear fa-spin')
  highlight_button = widgets.Button(


GridspecLayout(children=(HTML(value='<b>Export</b>', layout=Layout(grid_area='widget001', width='55px')), HTML…

Loaded from https://life.stg.wellzesta.com/login


## Playground

In [ ]:
start_playground()